In [ ]:
# from J. Beaumet


from cartopy import crs as ccrs, feature as cfeature

from osgeo import ogr

from shapely.geometry import shape

 

 

#shape_AURA='/bettik/jbeaumet-ext/REGION/POST-TRAITEMENTS/DONNEES_GEO/REGION_WGS84.shp'

shape_AURA='/mnt/dc5/data/DOSAT/DONNEES_GEOGRAPHIQUES/IGN/REGION_WGS84.shp'

file = ogr.Open(shape_AURA)

shape = file.GetLayer(0)

#first feature of the shapefile

feature = shape.GetFeature(10)

poly_AURA = feature.geometry()

region = feature.ExportToJson()

#print(region) # (GeoJSON format)

 

projet='EMIS_PACC-MACCS'

#path_out='/bettik/jbeaumet-ext/output/RESULTS_'+projet+'/'

path_MARCHIM='/bettik/jbeaumet-ext/output/'+projet+'/'

path_MARCHIM='/mnt/dc5/data/'+projet+'/'

var='PM25'        ### Improvement : put as args ###

dom_MARCHIM='EUR27KMext'

hlength='240'

 

YYYY=2015

date_debut=''+str(YYYY)+'010100'

date_fin=''+str(YYYY)+'011100'

 

print(path_MARCHIM)

 

# Create file with list of files to be read #

#os.system('ls '+path_MARCHIM+'out.'+str(date_debut)+'*_'+str(hlength)+'_'+dom_MARCHIM+'.nc')

filein=''+path_MARCHIM+'out.'+str(date_debut)+'_'+str(hlength)+'_'+dom_MARCHIM+'.nc'

 

with xr.open_dataset(filein) as ds:

               print(filein)

               lon1=ds['lon']

               lat1=ds['lat']

 

 

mask_AURA_27km=np.zeros((lon1.shape))

 

for y in range(0,lon1.shape[0]):

    for x in range(0,lon1.shape[1]):

        point = ogr.Geometry(ogr.wkbPoint)

        lon=float(lon1[y,x].data)

        lat=float(lat1[y,x].data)

        point.AddPoint_2D(lon, lat)

        if (poly_AURA.Contains(point)):

            mask_AURA_27km[y,x]=1

            print( lon, lat," point within AURA region")

 

fileout="mask_AURA_27km.npz"

np.savez(fileout, mask_AURA_27km=mask_AURA_27km, lon=lon1, lat=lat1)

############################################################################